In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
import optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [ ]:
TARGET = 'Survived'
train_df = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
submission = pd.read_csv('../input/81722/23_08_v5.csv')
test_df['Survived'] = submission['Survived']
all_df = pd.concat([train_df, test_df.sample(frac = 0.4)]).reset_index(drop=True)

In [ ]:
# Age fillna with mean age for each class
age_map = all_df[['Age', 'Pclass']].dropna().groupby('Pclass').mean().to_dict()
all_df.Age = all_df.Age.fillna(all_df.Pclass.map(age_map['Age']))

# Cabin, fillna with 'X' and take first letter
all_df.Cabin = all_df.Cabin.fillna('X').map(lambda x: x[0].strip())

# Ticket, fillna with 'X', split string and take first split 
all_df.Ticket = all_df.Ticket.fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

# Fare, fillna with mean value
fare_map = all_df[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
all_df['Fare'] = all_df['Fare'].fillna(all_df['Pclass'].map(fare_map['Fare']))
all_df['Fare'] = np.log1p(all_df['Fare'])

# Embarked, fillna with 'X' value
all_df.Embarked = all_df.Embarked.fillna('X')

# Name, take only surnames
all_df.Name = all_df.Name.map(lambda x: x.split(',')[0])
#family
all_df['FamilySize'] = all_df['SibSp'] + all_df['Parch'] + 1
all_df['IsAlone'] = np.where(all_df['FamilySize']<=1,1,0)
#cabin

all_df['Cabin'] = all_df["Cabin"].fillna('X').map(lambda x: x[0].strip())
all_df['Has_Cabin'] = all_df["Cabin"].apply(lambda x: 0 if (x != 'X') else 1)
cabin_map = {'A' : 1,'B' : 2, 'C' : 3, 'D' : 4, 'E' : 5, 'F' : 6, 'G' : 7, 'T' : 1, 'X': 8}
all_df['Cabin'] = all_df['Cabin'].str[0].fillna('X').replace(cabin_map)
#Embarked
all_df['Embarked'] = all_df['Embarked'].fillna("No")
conditions = [
    (all_df['Embarked']=="S"),
    (all_df['Embarked']=="Q"),
    (all_df['Embarked']=="C"),
    (all_df['Embarked']=="No")
]
choices = [0, 1, 2, -1]
all_df["Embarked"] = np.select(conditions, choices)
all_df['Embarked'] = all_df['Embarked'].astype(int)

In [ ]:
all_df

In [ ]:
label_cols = [ 'Ticket', 'Sex', 'IsAlone', 'Has_Cabin']
onehot_cols = ['Cabin', 'Embarked']
numerical_cols = ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'FamilySize']

In [ ]:
def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

scaler = StandardScaler()

onehot_encoded_df = pd.get_dummies(all_df[onehot_cols])
label_encoded_df = all_df[label_cols].apply(label_encoder)
numerical_df = pd.DataFrame(scaler.fit_transform(all_df[numerical_cols]), columns=numerical_cols)
target_df = all_df[TARGET]

all_df = pd.concat([label_encoded_df, numerical_df, onehot_encoded_df], axis=1)

In [ ]:
all_df

In [ ]:
train = all_df[:train_df.shape[0]]
target = train.pop('Survived')
test = all_df[train_df.shape[0]:]

In [ ]:
lgb_ft = train.columns

In [ ]:
train.head()

In [ ]:
def objective(trial,data=all_df,target=target_df):
    
    train_x, test_x, train_y, test_y = train_test_split(data[lgb_ft], target, test_size=0.15,random_state=42)
    params = {
        'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 11, 333),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.02, 0.05, 0.005, 0.1]),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 2000, 8000),
        'cat_smooth' : trial.suggest_int('cat_smooth', 10, 100),
        'cat_l2': trial.suggest_int('cat_l2', 1, 20),
        'min_data_per_group': trial.suggest_int('min_data_per_group', 50, 200),
        'cat_feature' : [0,1,2,3],
        'n_jobs' : -1, 
        'random_state': 26,
        'boosting_type': 'gbdt',
        'metric': 'binary_logloss',
        'objective' : 'binary',
        'device': 'gpu'
    }
    model = LGBMClassifier(**params)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)], early_stopping_rounds=300, verbose=False)
    
    preds = model.predict(test_x)
    
    acc = accuracy_score(test_y , preds)
    
    return acc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
params = study.best_trial.params

In [ ]:
params['random_state'] = 26
params['device'] = 'gpu'
params['n_jobs'] = -1 
params['cat_feature'] = [0,1,2,3]
params['boosting_type'] =  'gbdt'
params['metric'] =  'binary_logloss'
params['objective'] = 'binary'
params

In [ ]:
NFOLDS = 2
folds = StratifiedKFold(n_splits=NFOLDS, random_state=26, shuffle=True)
splits = folds.split(all_df, target_df)
lgb_train_preds = np.zeros(all_df.shape[0])
lgb_test_preds = np.zeros(test.shape[0])
for fold, (train_index, test_index) in enumerate(splits):
    print("--> Fold {}".format(fold + 1))
    
    X_train, X_valid = all_df[lgb_ft].iloc[train_index], all_df[lgb_ft].iloc[test_index]
    y_train, y_valid = target_df.iloc[train_index], target_df.iloc[test_index]
    pre_model = LGBMClassifier(**params).fit(X_train, y_train, 
                                                  eval_set=[(X_valid, y_valid)],  
                                                  early_stopping_rounds=300, verbose = 0)
    params2 = params.copy()
    params2['learning_rate'] = params['learning_rate']*0.1
    model = LGBMClassifier(**params).fit(X_train, y_train, 
                                                  eval_set=[(X_valid, y_valid)],  
                                                  early_stopping_rounds=300, verbose = 0,
                                                  init_model = pre_model)
    train_oof_preds = model.predict(X_valid)
    test_oof_preds = model.predict(test[lgb_ft])
    lgb_train_preds[test_index] = train_oof_preds
    lgb_test_preds += test_oof_preds / NFOLDS
    print(": LGB - Accuracy Score = {}".format(accuracy_score(y_valid, train_oof_preds)))

print("--> Overall metrics")
print(": LGB - Accuracy Score = {}".format(accuracy_score(target_df, lgb_train_preds)))

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')
sub['Survived'] = np.round(lgb_test_preds).astype(int)
regr_table = sub.to_csv("25_04_v1.csv", index = False)